In [59]:
import pandas as pd

dec_merged = pd.read_csv('../DATASET/obs_est_merged/dec_merged.csv')

In [60]:
dec_merged

,lat,lon,year,month,precip_est,precip_obs,bias_dec
0,45.0,-20.0,1982,1,114.242190,158.107760,-43.865570
1,45.0,-19.0,1982,1,109.765625,138.565060,-28.799435
2,45.0,-18.0,1982,1,106.218750,122.867584,-16.648834
3,45.0,-17.0,1982,1,105.335940,109.741210,-4.405270
4,45.0,-16.0,1982,1,103.375000,106.399536,-3.024536
...,...,...,...,...,...,...,...
230251,20.0,16.0,2017,6,0.351885,3.776550,-3.424665
230252,20.0,17.0,2017,6,0.426104,3.719330,-3.293226
230253,20.0,18.0,2017,6,1.090166,3.147125,-2.056959
230254,20.0,19.0,2017,6,1.437823,0.000000,1.437823


In [61]:
dec_merged = dec_merged.drop(columns=['year'])

dec_average= dec_merged.groupby(['month', 'lat', 'lon']).agg({'precip_est': 'mean', 'precip_obs': 'mean', 'bias_dec': 'mean'}).reset_index()
dec_average['is_train'] = dec_average['month'].apply(lambda x: x in [1, 2, 3])  # temporary splitting
dec_average = dec_average.drop(columns=['precip_obs'])

In [62]:
dec_average

,month,lat,lon,precip_est,bias_dec,is_train
0,1,20.0,-20.0,2.776387,0.318981,True
1,1,20.0,-19.0,2.335524,-0.617370,True
2,1,20.0,-18.0,1.751973,-0.597668,True
3,1,20.0,-17.0,1.510762,-1.134836,True
4,1,20.0,-16.0,1.556878,-2.018235,True
...,...,...,...,...,...,...
6391,6,45.0,16.0,108.792222,23.485736,False
6392,6,45.0,17.0,97.997190,15.645188,False
6393,6,45.0,18.0,90.076890,17.988436,False
6394,6,45.0,19.0,82.176879,19.693621,False


In [67]:
dec_average[dec_average['is_train']].describe()

,month,lat,lon,precip_est,bias_dec
count,3198.000000,3198.000000,3198.00000,3198.000000,3198.000000
mean,2.000000,32.500000,0.00000,36.607567,6.695936
std,0.816624,7.501173,11.83401,36.744623,14.831957
min,1.000000,20.000000,-20.00000,0.036804,-89.495418
25%,1.000000,26.000000,-10.00000,2.076757,-0.369929
50%,2.000000,32.500000,0.00000,28.161892,1.149002
75%,3.000000,39.000000,10.00000,65.208199,15.642412
max,3.000000,45.000000,20.00000,187.764559,70.570354


In [68]:
dec_average[~dec_average['is_train']].describe()

,month,lat,lon,precip_est,bias_dec
count,3198.000000,3198.000000,3198.00000,3198.000000,3198.000000
mean,5.000000,32.500000,0.00000,24.220307,9.442057
std,0.816624,7.501173,11.83401,28.254272,13.934988
min,4.000000,20.000000,-20.00000,0.107813,-108.419120
25%,4.000000,26.000000,-10.00000,1.150145,0.346239
50%,5.000000,32.500000,0.00000,11.061715,3.791268
75%,6.000000,39.000000,10.00000,42.108622,17.708777
max,6.000000,45.000000,20.00000,154.108844,73.756035


In [73]:
train = dec_average[dec_average['is_train']]
test = dec_average[~dec_average['is_train']]

train = train.sample(frac=1, random_state=42)
X, y = train[['lat', 'lon', 'month', 'precip_est']], train['bias_dec'] 

### Model selection

In [74]:
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold

In [78]:
# Define the models
models = {
    "Linear Regression": LinearRegression(),
    "XGBoost": XGBRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
    "ExtraTrees Regressor": ExtraTreesRegressor(random_state=42),
    "LGBM Regressor": LGBMRegressor(random_state=42)
}
k = 4
kf = KFold(n_splits=k)
results = {}
for model_name, model in models.items():
    train_rmse_scores = []
    test_rmse_scores = []

    for train_index, test_index in kf.split(X):
        x_train_fold, x_test_fold = X.iloc[train_index], X.iloc[test_index]
        y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

        model.fit(x_train_fold, y_train_fold)
        y_pred_train = model.predict(x_train_fold)
        y_pred_test = model.predict(x_test_fold)

        rmse_train = mean_squared_error(y_train_fold, y_pred_train, squared=False)
        rmse_test = mean_squared_error(y_test_fold, y_pred_test, squared=False)

        train_rmse_scores.append(rmse_train)
        test_rmse_scores.append(rmse_test)

    avg_train_rmse = sum(train_rmse_scores) / k
    avg_test_rmse = sum(test_rmse_scores) / k

    results[model_name] = {
        "train_rmse": avg_train_rmse,
        "test_rmse": avg_test_rmse,
    }

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001031 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 327
[LightGBM] [Info] Number of data points in the train set: 2398, number of used features: 4
[LightGBM] [Info] Start training from score 6.976365
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000048 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 327
[LightGBM] [Info] Number of data points in the train set: 2398, number of used features: 4
[LightGBM] [Info] Start training from score 6.590744
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000160 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 327

In [80]:
import plotly.graph_objects as go
import numpy as np
import plotly.io as pio

data = results
models = list(data.keys())
train_rmse = [data[model]['train_rmse'] for model in models]
test_rmse = [data[model]['test_rmse'] for model in models]

train_rmse = [round(num, 2) for num in train_rmse]
test_rmse = [round(num, 2) for num in test_rmse]

standard_deviation = np.std(y)  # Calculate standard deviation using numpy
sample_size = len(y)  # Calculate sample size

standard_error = standard_deviation / np.sqrt(sample_size)

In [81]:
fig = go.Figure()

# Bar chart for RMSE
fig.add_trace(go.Bar(
    x=models,
    y=train_rmse,
    name='Train RMSE',
    marker_color='blue',
    text=train_rmse,  # Add this line to specify the text for each bar
    # 'auto' places the text inside the bars; you can also use 'outside' or 'inside'
    textposition='auto'
))

fig.add_trace(go.Bar(
    x=models,
    y=test_rmse,
    name='Test RMSE',
    marker_color='red',
    text=test_rmse,  # Add this line to specify the text for each bar
    # 'auto' places the text inside the bars; you can also use 'outside' or 'inside'
    textposition='auto'
))
# Update the layout
fig.update_layout(
    barmode='group',
    title='RMSE',
    xaxis_title='Models',
    yaxis_title='Value',
    legend_title='Data',
    width=600,
    # plot_bgcolor='rgba(0,0,0,0)',  # Set plot background color to transparent
    # paper_bgcolor='rgba(0,0,0,0)'
)

# # Line chart for std
# fig.add_trace(go.Scatter(
#     x=models,
#     y=[stdev for model in models],
#     mode='lines+markers',
#     name='Std',
#     line=dict(color='green', width=2)
# ))

# Line chart for std
fig.add_trace(go.Scatter(
    x=models,
    y=[standard_deviation for i in range(len(models))],
    mode='lines+markers',
    name='Std',
    line=dict(color='orange', width=2)
))
fig.show()

### Hyperparams tuning

In [83]:
##  of XGboost
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np

# Define your model
xgb = XGBRegressor(random_state=42)

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# Setup GridSearchCV
scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=False)
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, scoring=scorer, cv=4, verbose=2)

# Split your data if not already split
# Assuming 'X' and 'y' are your features and target variable from your earlier split
grid_search.fit(X, y)

# Get the best estimator and its parameters
best_xgb = grid_search.best_estimator_
best_params = grid_search.best_params_

print("Best parameters:", best_params)
print("Best RMSE:", -grid_search.best_score_)  # Note: 'best_score_' is negative, so take the negative of it

# Optionally, you can use the best model to make predictions or further analysis
# predictions = best_xgb.predict(X_test)
# rmse = np.sqrt(mean_squared_error(y_test, predictions))
# print("Test RMSE:", rmse)

Fitting 4 folds for each of 243 candidates, totalling 972 fits
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.9; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.9; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.9; total time=   0.0s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.9; total time=   0.0s
[CV] END 

In [90]:
from bayes_opt import BayesianOptimization
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

# Define your XGBoost training function
def xgb_evaluate(max_depth, learning_rate, n_estimators, subsample, colsample_bytree):
    params = {
        'max_depth': int(max_depth),
        'learning_rate': learning_rate,
        'n_estimators': int(n_estimators),
        'subsample': subsample,
        'colsample_bytree': colsample_bytree,
        'random_state': 42
    }
    xgb = XGBRegressor(**params)
    # Change 'cv' to a higher number for better accuracy but longer runtime
    cv_scores = cross_val_score(xgb, X, y, cv=4, scoring='neg_root_mean_squared_error')
    return np.mean(cv_scores)

# Set up Bayesian Optimization
optimizer = BayesianOptimization(
    f=xgb_evaluate,
    pbounds={
        'max_depth': (3, 7),
        'learning_rate': (0.1, 0.12),
        'n_estimators': (100, 300),
        'subsample': (0.8, 1.0),
        'colsample_bytree': (0.9, 1.0)
    },
    random_state=42
)

# Run optimization
optimizer.maximize(init_points=10, n_iter=70)

# Print best parameters
print("Best parameters:", optimizer.max['params'])

# You may need to round or convert the parameters as needed to fit into the model, e.g., max_depth should be an integer.


|   iter    |  target   | colsam... | learni... | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------------------
| 1         | -7.84     | 0.9375    | 0.119     | 5.928     | 219.7     | 0.8312    |
| 2         | -7.556    | 0.9156    | 0.1012    | 6.465     | 220.2     | 0.9416    |
| 3         | -7.682    | 0.9021    | 0.1194    | 6.33      | 142.5     | 0.8364    |
| 4         | -7.905    | 0.9183    | 0.1061    | 5.099     | 186.4     | 0.8582    |
| 5         | -8.242    | 0.9612    | 0.1028    | 4.169     | 173.3     | 0.8912    |
| 6         | -7.9      | 0.9785    | 0.104     | 5.057     | 218.5     | 0.8093    |
| 7         | -8.755    | 0.9608    | 0.1034    | 3.26      | 289.8     | 0.9931    |
| 8         | -8.781    | 0.9808    | 0.1061    | 3.391     | 236.8     | 0.888     |
| 9         | -8.698    | 0.9122    | 0.1099    | 3.138     | 281.9     | 0.8518    |
| 10        | -7.832    | 0.9663    | 0.1062    | 5.08

In [91]:
best_params = {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 300, 'subsample': 0.8}
# Create Extra Trees model using the best parameters
best_model = XGBRegressor(**best_params, random_state=42)

In [93]:
k = 4
kf = KFold(n_splits=k)
final_results = {}

train_rmse_scores = []
test_rmse_scores = []

for train_index, test_index in kf.split(X):
    # Use .iloc for positional indexing
    x_train_fold, x_test_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

    best_model.fit(x_train_fold, y_train_fold)
    y_pred_train = best_model.predict(x_train_fold)
    y_pred_test = best_model.predict(x_test_fold)

    rmse_train = mean_squared_error(
        y_train_fold, y_pred_train, squared=False)
    rmse_test = mean_squared_error(y_test_fold, y_pred_test, squared=False)

    train_rmse_scores.append(rmse_train)
    test_rmse_scores.append(rmse_test)

avg_train_rmse = sum(train_rmse_scores) / k
avg_test_rmse = sum(test_rmse_scores) / k

final_results["metrics"] = {
    "RMSE train": avg_train_rmse,
    "RMSE test": avg_test_rmse,
}
print(final_results)

{'metrics': {'RMSE train': 0.6107314494222817, 'RMSE test': 7.252401902448559}}


In [94]:
X_sub = test[['lat', 'lon', 'month', 'precip_est']]

y_sub = best_model.predict(X_sub)

In [95]:
rmse = np.sqrt(mean_squared_error(test['bias_dec'], y_sub))

In [96]:
rmse

17.677579231440316